In [2]:
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"

from transformers import BertModel, BertTokenizer
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import ast
from sklearn.model_selection import train_test_split
from torch.optim import AdamW
# import matplotlib.pyplot as plt

from tqdm import tqdm

In [3]:
# Import the dataset
df = pd.read_csv("final_resume_dataset.csv")
print(df.head())

         Resume Job Role                                  Resume Experience  \
0      Market Researcher  ['Conducted surveys and focus groups to gather...   
1  Research Psychologist  ['Developed and conducted surveys to assess us...   
2      Senior Researcher  ['Assisted in the design and execution of in-d...   
3             QA Manager  ['Managed and maintained office facilities, in...   
4      Logistics Manager  ['Developed and executed space planning strate...   

                                       Resume Skills           Job Title  \
0  Market research tools and techniques User inte...     User Researcher   
1  Research methodology Usability testing with in...     User Researcher   
2  Social science research methods User testing S...     User Researcher   
3  Construction management Estate planning Enviro...  Facilities Manager   
4  Project management Server maintenance Site pla...  Facilities Manager   

                                     Job Description  \
0  Conduct u

In [4]:
# Load the bert tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [5]:
class FieldPairDataset(Dataset):
    def __init__(self, df, tokenizer, field_1, field_2, label_field, max_len=512, combine_fields=None):
        self.df = df
        self.tokenizer = tokenizer
        self.field_1 = field_1
        self.field_2 = field_2
        self.label_field = label_field
        self.max_len = max_len
        self.combine_fields = combine_fields  # optional tuple of fields to join

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        # Support concatenating multiple fields into input_a
        if self.combine_fields:
            input_a = " ".join(str(row[f]) for f in self.combine_fields)
        else:
            input_a = str(row[self.field_1])

        input_b = str(row[self.field_2])
        label = float(row[self.label_field])

        # Pretty-print list fields like Resume Experience
        try:
            if input_b.startswith("[") and isinstance(eval(input_b), list):
                input_b = "\n- " + "\n- ".join(ast.literal_eval(input_b))
        except:
            pass

        encoding = self.tokenizer(
            input_a,
            input_b,
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors="pt"
        )

        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "label": torch.tensor(label, dtype=torch.float)
        }


In [6]:
# Example usage of the dataset
# Title Match
title_dataset = FieldPairDataset(
    df,
    tokenizer,
    field_1="Job Title",
    field_2="Resume Job Role",
    label_field="Job Title Match"
)

# Skills Match
skills_dataset = FieldPairDataset(
    df,
    tokenizer,
    field_1="Job Skills",
    field_2="Resume Skills",
    label_field="Skills Match"
)

# Responsibilities Match (combining Job Description + Responsibilities)
responsibilities_dataset = FieldPairDataset(
    df,
    tokenizer,
    field_1=None,
    field_2="Resume Experience",
    label_field="Responsibilities Match",
    combine_fields=("Job Description", "Job Responsibilities")
)


In [7]:
# Make a dataloader with proper train test splits
# df = df.sample(frac=0.1, random_state=42)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Title Match
title_train_dataset = FieldPairDataset(train_df, tokenizer,
    field_1="Job Title", field_2="Resume Job Role", label_field="Job Title Match")

title_test_dataset = FieldPairDataset(test_df, tokenizer,
    field_1="Job Title", field_2="Resume Job Role", label_field="Job Title Match")

title_train_loader = DataLoader(title_train_dataset, batch_size=8, shuffle=True)
title_test_loader = DataLoader(title_test_dataset, batch_size=8, shuffle=False)

# Skills Match
skills_train_dataset = FieldPairDataset(train_df, tokenizer,
    field_1="Job Skills", field_2="Resume Skills", label_field="Skills Match")

skills_test_dataset = FieldPairDataset(test_df, tokenizer,
    field_1="Job Skills", field_2="Resume Skills", label_field="Skills Match")

skills_train_loader = DataLoader(skills_train_dataset, batch_size=8, shuffle=True)
skills_test_loader = DataLoader(skills_test_dataset, batch_size=8, shuffle=False)

# Responsibilities Match
resp_train_dataset = FieldPairDataset(train_df, tokenizer,
    field_1=None, field_2="Resume Experience", label_field="Responsibilities Match",
    combine_fields=("Job Description", "Job Responsibilities"))

resp_test_dataset = FieldPairDataset(test_df, tokenizer,
    field_1=None, field_2="Resume Experience", label_field="Responsibilities Match",
    combine_fields=("Job Description", "Job Responsibilities"))

resp_train_loader = DataLoader(resp_train_dataset, batch_size=8, shuffle=True)
resp_test_loader = DataLoader(resp_test_dataset, batch_size=8, shuffle=False)

In [8]:
# Make the model
class JobFitModel(nn.Module):
    def __init__(self):
        super(JobFitModel, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.ffn = nn.Sequential(
            nn.Linear(768, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )


    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_embedding = outputs.last_hidden_state[:, 0, :]  # Grab [CLS] token
        score = self.ffn(cls_embedding)
        return score.squeeze(1)  # Return shape: (batch_size,)

In [9]:
def train_model(model, train_loader, optimizer, criterion, device, epochs=3):
    model.to(device)

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        loop = tqdm(train_loader, desc=f"Epoch {epoch+1}")
        for batch in loop:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)

            optimizer.zero_grad()

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            loop.set_postfix(loss=loss.item())

        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1} training loss: {avg_loss:.4f}")


In [10]:
def evaluate_model(model, test_loader, criterion, device):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            all_preds.extend(outputs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_loss = total_loss / len(test_loader)
    print(f"Test MSE Loss: {avg_loss:.4f}")
    return all_preds, all_labels


In [11]:
def plot_predictions(preds, labels):
    plt.figure(figsize=(6, 6))
    plt.scatter(labels, preds, alpha=0.5, color='blue')
    plt.plot([0, 1], [0, 1], linestyle='--', color='red')  # ideal diagonal
    plt.xlabel("True Suitability Score")
    plt.ylabel("Predicted Score")
    plt.title("Predicted vs. True Suitability")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [12]:
def predict_suitability(model, tokenizer, jd_text, resume_text, device):
    model.eval()

    encoding = tokenizer(
        jd_text, resume_text,
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=512
    )

    input_ids = encoding["input_ids"].to(device)
    attention_mask = encoding["attention_mask"].to(device)

    with torch.no_grad():
        output = model(input_ids=input_ids, attention_mask=attention_mask)

    return output.item()


In [13]:
# Setup variables
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [14]:
# Setup all models
title_model = JobFitModel().to(device)
title_optimizer = torch.optim.AdamW(title_model.parameters(), lr=2e-5)
title_criterion = nn.MSELoss()

skills_model = JobFitModel().to(device)
skills_optimizer = torch.optim.AdamW(skills_model.parameters(), lr=2e-5)
skills_criterion = nn.MSELoss()

resp_model = JobFitModel().to(device)
resp_optimizer = torch.optim.AdamW(resp_model.parameters(), lr=2e-5)
resp_criterion = nn.MSELoss()


In [17]:
# Train title match model
train_model(title_model, title_train_loader, title_optimizer, title_criterion, device)

Epoch 1:   1%|          | 32/4233 [00:06<15:18,  4.57it/s, loss=0.0141]


KeyboardInterrupt: 

In [ ]:
# Train skills match model
train_model(skills_model, skills_train_loader, skills_optimizer, skills_criterion, device)

In [ ]:
# Train responsibilities match model
train_model(resp_model, resp_train_loader, resp_optimizer, resp_criterion, device)

In [15]:
# Evaluate Model
# Evaluate Title Match
title_preds, title_labels = evaluate_model(title_model, title_test_loader, title_criterion, device)
plot_predictions(title_preds, title_labels)

# Evaluate Skills Match
skills_preds, skills_labels = evaluate_model(skills_model, skills_test_loader, skills_criterion, device)
plot_predictions(skills_preds, skills_labels)

# Evaluate Responsibilities Match
resp_preds, resp_labels = evaluate_model(resp_model, resp_test_loader, resp_criterion, device)
plot_predictions(resp_preds, resp_labels)


KeyboardInterrupt: 

In [16]:
# Use the model
# Job Posting Fields
job_title = "User Researcher"
job_skills = "User research methods, usability testing, data analysis"
job_responsibilities = "Conduct interviews and usability studies. Synthesize user insights."
job_description = "Understand user behavior and feedback."

# Resume Fields
resume_role = "Market Researcher"
resume_skills = "Qualitative research, A/B testing, surveys"
resume_experience = "- Conducted focus groups\n- Analyzed survey data\n- Presented user insights"

# Title Match Prediction
title_score = predict_suitability(
    title_model,
    tokenizer,
    jd_text=job_title,
    resume_text=resume_role,
    device=device
)

# Skills Match Prediction
skills_score = predict_suitability(
    skills_model,
    tokenizer,
    jd_text=job_skills,
    resume_text=resume_skills,
    device=device
)

# Responsibilities Match Prediction
responsibilities_input = job_responsibilities + " " + job_description
resp_score = predict_suitability(
    resp_model,
    tokenizer,
    jd_text=responsibilities_input,
    resume_text=resume_experience,
    device=device
)
print(f"Title Match Score:           {title_score:.3f}")
print(f"Skills Match Score:          {skills_score:.3f}")
print(f"Responsibilities Match Score:{resp_score:.3f}")

overall_score = (title_score + skills_score + resp_score) / 3
print(f"\nOverall Predicted Suitability: {overall_score:.3f}")


Title Match Score:           0.524
Skills Match Score:          0.487
Responsibilities Match Score:0.490

Overall Predicted Suitability: 0.500


In [ ]:
torch.save(title_model.state_dict(), "title_model.pt")
torch.save(skills_model.state_dict(), "skills_model.pt")
torch.save(resp_model.state_dict(), "resp_model.pt")


In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

title_model = JobFitModel()
skills_model = JobFitModel()
resp_model = JobFitModel()

title_model.load_state_dict(torch.load("title_model (1).pt", map_location=device))
skills_model.load_state_dict(torch.load("skills_model (1).pt", map_location=device))
resp_model.load_state_dict(torch.load("resp_model (1).pt", map_location=device))

title_model.to(device).eval()
skills_model.to(device).eval()
resp_model.to(device).eval()

job_title = "Head Baker"
job_skills = "bake cookies, serve customers food"
job_responsibilities = "Use a lot of ovens and make cookie batter"
job_description = "Lead chefs to make lots of baked goods."

resume_role = "Head Baker"
resume_skills = "Bakes good cookies, and muffins"
resume_experience = "- Lead two chefs to make 400 cookies every day"

title_score = predict_suitability(
    title_model,
    tokenizer,
    jd_text=job_title,
    resume_text=resume_role,
    device=device
)

skills_score = predict_suitability(
    skills_model,
    tokenizer,
    jd_text=job_skills,
    resume_text=resume_skills,
    device=device
)

responsibilities_input = job_responsibilities + " " + job_description
resp_score = predict_suitability(
    resp_model,
    tokenizer,
    jd_text=responsibilities_input,
    resume_text=resume_experience,
    device=device
)

print(f"Title Match Score:           {title_score:.3f}")
print(f"Skills Match Score:          {skills_score:.3f}")
print(f"Responsibilities Match Score:{resp_score:.3f}")

overall_score = (title_score + skills_score + resp_score) / 3
print(f"\nOverall Predicted Suitability: {overall_score:.3f}")


Title Match Score:           0.949
Skills Match Score:          0.696
Responsibilities Match Score:0.672

Overall Predicted Suitability: 0.772


In [24]:
import pandas as pd

# Load your CSV
df = pd.read_csv("combined_data.csv")
df.columns = df.columns.str.strip()

# Create empty columns to store scores
df["Title Match Score"] = 0.0
df["Skills Match Score"] = 0.0
df["Responsibilities Match Score"] = 0.0
df["Overall Suitability Score"] = 0.0

# Loop through each row and compute scores
for i, row in df.iterrows():
    job_title = str(row["job_role"])
    job_skills = str(row["job_skills"])
    job_responsibilities = str(row["job_responsibilities"])
    job_description = str(row["job_description"])

    resume_role = str(row["applicant_job_role"])
    resume_skills = str(row["applicant_skills"])
    resume_experience = str(row["applicant_experience"])

    responsibilities_input = job_responsibilities + " " + job_description

    title_score = predict_suitability(title_model, tokenizer, job_title, resume_role, device)
    skills_score = predict_suitability(skills_model, tokenizer, job_skills, resume_skills, device)
    resp_score = predict_suitability(resp_model, tokenizer, responsibilities_input, resume_experience, device)

    df.at[i, "Title Match Score"] = title_score
    df.at[i, "Skills Match Score"] = skills_score
    df.at[i, "Responsibilities Match Score"] = resp_score
    df.at[i, "Overall Suitability Score"] = (title_score + skills_score + resp_score) / 3

# View the results
print(df.head())


                                  applicant_job_role  \
0  Numerated, Fintech Startup Acquired by Moody’s...   
1                                 Data Science Co-op   
2                            Software Engineer Co-op   

                                applicant_experience  \
0  ['Designed and implemented a portfolio reporti...   
1  ['Developed, tested and optimized machine lear...   
2  ['Converting APIframeworkfrom Meteor to Expres...   

                                    applicant_skills           job_role  \
0  Languages: Javascript, Python, Typescript, Jav...  Software Engineer   
1  Languages: Python, C, Java, Typescript, JavaSc...  Software Engineer   
2  Languages: Python, TypeScript, JavaScript, Jav...  Software Engineer   

                                     job_description  \
0  We are looking for a Software Engineer to desi...   
1  We are looking for a Software Engineer to desi...   
2  We are looking for a Software Engineer to desi...   

                        